In [47]:
!pip install numpy pandas matplotlib seaborn scikit-learn --quiet
!pip install scikit-learn xgboost --upgrade --quiet
# !pip install opendatasets --quiet
!pip install plotly --quiet
# import opendatasets as od
import pandas as pd
import numpy as np
# dataset_url = 'https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques'
# od.download(dataset_url)

Shape of data


In [48]:
data_dir = './data'
data_df = pd.read_csv(f'{data_dir}/train.csv', nrows=100000)
test_df = pd.read_csv(f'{data_dir}/test.csv', nrows=100000)
data_df.shape

(1460, 81)

In [49]:
import plotly.express as px

# compute correlations with SalePrice
corr_series = data_df.corr(numeric_only=True)['SalePrice'].drop('SalePrice')
corr_df = corr_series.reset_index().rename(columns={'index':'feature', 'SalePrice':'corr'})

# sort by absolute correlation so strongest dependencies appear first
corr_df = corr_df.sort_values('corr', key=lambda x: x.abs(), ascending=False)

# bar chart (signed correlation, colored by sign/magnitude)
fig = px.bar(
    corr_df,
    x='feature',
    y='corr',
    color='corr',
    color_continuous_scale='RdBu',
    title='Correlation of features with SalePrice',
    labels={'corr':'corr_with_SalePrice', 'feature':'feature'}
)
fig.update_layout(yaxis=dict(range=[-1,1]), xaxis_title='Feature', yaxis_title='Correlation')
fig.show(renderer="browser")

//Remove Field are BsmtFinType2 , BsmtHalfBath,MiscVal,Id,LowQualFinSF,YrSold,MoSold,3SsnPorch

In [50]:
remove_cols = ['BsmtFinType2', 'BsmtHalfBath', 'MiscVal', 'Id', 'LowQualFinSF', 'YrSold', 'MoSold', '3SsnPorch']

test_remove_cols = ['BsmtFinType2', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold', 'MoSold', '3SsnPorch']





In [51]:
data_df = data_df.drop(remove_cols, axis=1)
test_df = test_df.drop(test_remove_cols,axis=1)

In [52]:
# Show number of nulls per column
def checkNullValues(df):
    null_counts = df.isnull().sum().sort_values(ascending=False)
    print(null_counts[null_counts > 0])


MasVnrType = None 
MasVnrArea,GarageYrBlt , LotFrontage = Numerical
Electrical  drop it 


In [53]:
def clean_missing_values(df):
    # Drop 'Electrical'
    if 'Electrical' in df.columns:
        df.drop('Electrical', axis=1, inplace=True)
    
    # Fill MasVnrType with 'None'
    if 'MasVnrType' in df.columns:
        df['MasVnrType'].fillna('None', inplace=True)
    
    # Columns to exclude from filling (numerical columns)
    exclude_cols = ['MasVnrArea', 'GarageYrBlt', 'LotFrontage', 'MasVnrType']
    
    # Fill other categorical nulls with 'NA'
    for col in df.select_dtypes(include='object').columns:
        if col not in exclude_cols:
            df[col].fillna('NA', inplace=True)
    
    return df




In [54]:
data_df = clean_missing_values(data_df)
test_df = clean_missing_values(test_df)

/tmp/ipykernel_7171/2073745228.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_7171/2073745228.py:16: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' o

In [55]:
def most_Val(df, col_name):
    """
    Returns the most common (mode) value of a numerical column.
    """
    if col_name in df.columns:
        # Drop nulls first
        mode_value = df[col_name].mode(dropna=True)
        if not mode_value.empty:
            return mode_value[0]
        else:
            return None
    else:
        return None

# Usage for GarageYrBlt
common_year = most_Val(data_df, 'GarageYrBlt')
print("Most common Garage Year Built:", common_year)
if common_year is not None:
    data_df['GarageYrBlt'].fillna(common_year, inplace=True)


Most common Garage Year Built: 2005.0


/tmp/ipykernel_7171/1016363599.py:19: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [56]:
checkNullValues(data_df)


LotFrontage    259
MasVnrArea       8
dtype: int64


In [57]:
area1 = most_Val(data_df, 'MasVnrArea')
print("Most common MasVnrArea:", area1)
data_df['MasVnrArea'].fillna(area1, inplace=True)
area2 = most_Val(test_df, 'MasVnrArea')
print("Most common MasVnrArea in test set:", area2)
test_df['MasVnrArea'].fillna(area2, inplace=True)

Most common MasVnrArea: 0.0


/tmp/ipykernel_7171/2560257206.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Most common MasVnrArea in test set: 0.0


/tmp/ipykernel_7171/2560257206.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [58]:
data_df
area1 = most_Val(data_df, 'LotFrontage')
area1
data_df['LotFrontage'].fillna(area1, inplace=True)
area2 = most_Val(test_df, 'LotFrontage')
test_df['LotFrontage'].fillna(area2, inplace=True)

/tmp/ipykernel_7171/2058303115.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_7171/2058303115.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or

input & catogrical cols

In [59]:
def Input_Categorical_Cols(df,numerical_cols, remove_cols=None, show_summary=True):
    # handle case if remove_cols not provided
    if remove_cols is None:
        remove_cols = []

    output_col = 'SalePrice'
    
    # input columns excluding output_col
    input_col = [col for col in df.columns if col != output_col]

    # predefined numerical columns
   

    # remove excluded columns from numerical
    numerical_cols = [col for col in numerical_cols if col in df.columns and col not in remove_cols and col != output_col]

    # remaining input columns → categorical
    categorical_cols = [
        col for col in input_col
        if col not in numerical_cols and col not in remove_cols and col != output_col
    ]

    # Optional summary
    if show_summary:
        print(f"🧮 Total Columns: {len(df.columns)}")
        print(f"📊 Numerical Columns: {len(numerical_cols)}")
        print(f"🔤 Categorical Columns: {len(categorical_cols)}")
        print(f"📋 List of Categorical Columns:\n{categorical_cols}")

    return input_col, output_col, numerical_cols, categorical_cols


In [60]:
numerical_cols = [
        'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
        'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
        '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
        'FullBath', 'HalfBath', 'Bedroom', 'Kitchen', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'
    ]
test_numerical_cols =  [
        'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
        'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
        '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
        'FullBath', 'HalfBath', 'Bedroom', 'Kitchen', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal','Id'
    ]
input_col, output_col, numerical_cols, categorical_cols = Input_Categorical_Cols(data_df,numerical_cols);
test_input_col, test_output_col, test_numerical_cols, test_categorical_cols = Input_Categorical_Cols(test_df,test_numerical_cols);

🧮 Total Columns: 72
📊 Numerical Columns: 25
🔤 Categorical Columns: 46
📋 List of Categorical Columns:
['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'HeatingQC', 'CentralAir', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
🧮 Total Columns: 72
📊 Numerical Columns: 26
🔤 Categorical Columns: 46
📋 List of Categorical Columns:
['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'Hou

Copy data 

In [61]:
train_df = data_df.copy()

OneHotEncoding  

View Categorial_col

In [62]:
import plotly.express as px
import pandas as pd

def plot_categorical_overview(df, categorical_cols):
    """
    Plots a single bar chart showing how many unique values each categorical column has.
    Highlights the column with the most categories.
    """
    # Calculate unique counts
    unique_counts = {col: df[col].nunique() for col in categorical_cols if col in df.columns}
    data = pd.DataFrame(list(unique_counts.items()), columns=['Column', 'Unique_Values'])

    # Find the column with the highest unique values
    max_val = data['Unique_Values'].max()
    data['Highlight'] = data['Unique_Values'].apply(lambda x: 'Most Unique' if x == max_val else 'Other')

    # Create bar chart
    fig = px.bar(
        data,
        x='Column',
        y='Unique_Values',
        color='Column',  # Each bar gets a unique color
        title='Unique Value Count per Categorical Column',
        text='Unique_Values',
        color_discrete_sequence=px.colors.qualitative.Vivid
    )

    # Add highlight effect for the most unique column
    fig.update_traces(
        marker_line_width=1.5,
        marker_line_color='black',
        textposition='outside'
    )

    # Highlight the bar with max value
    fig.update_traces(
        marker_line_width=2.5,
        marker_line_color=['red' if val == max_val else 'black' for val in data['Unique_Values']]
    )

    # Layout tweaks
    fig.update_layout(
        xaxis_title="Categorical Columns",
        yaxis_title="Number of Unique Values",
        showlegend=False,
        template='plotly_white',
        width=950,
        height=500
    )

    fig.show(renderer="browser")


In [63]:
plot_categorical_overview(train_df, categorical_cols)

function

In [64]:
import pandas as pd

def one_hot_encode_all(df, categorical_cols, drop_first=False):
    """
    Performs one-hot encoding on all specified categorical columns and removes original columns.
    
    Parameters:
        df (pd.DataFrame): Input dataframe.
        categorical_cols (list): List of categorical column names.
        drop_first (bool): Whether to drop the first dummy variable to avoid multicollinearity.
        
    Returns:
        pd.DataFrame: New dataframe with one-hot encoded categorical columns.
    """
    # Ensure only valid columns
    categorical_cols = [col for col in categorical_cols if col in df.columns]
    
    # Perform one-hot encoding (original columns are removed automatically)
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=drop_first, dtype=int)
    
    print(f"✅ One-hot encoding applied to {len(categorical_cols)} columns.")
    print(f"🔢 Data shape changed from {df.shape} → {df_encoded.shape}")
    
    return df_encoded


In [65]:

# Call the function
df_encoded = one_hot_encode_all(train_df, categorical_cols)
test_df_encoded = one_hot_encode_all(test_df, test_categorical_cols)

✅ One-hot encoding applied to 46 columns.
🔢 Data shape changed from (1460, 72) → (1460, 326)
✅ One-hot encoding applied to 46 columns.
🔢 Data shape changed from (1459, 72) → (1459, 315)


In [66]:
input_col = [col for col in df_encoded.columns if col != output_col]
test_input_col = [col for col in test_df_encoded.columns if col != test_output_col]
print(len(input_col))
print(len(test_input_col))

325
315


HardCoded Model

In [67]:
import numpy as np

class MeanRegressor:
    def fit(self, X, y):
        # y must be a numeric array or pandas Series
        self.mean = y.mean()

    def predict(self, X):
        return np.full(X.shape[0], self.mean)


In [68]:
mean_model = MeanRegressor()

In [69]:
X = df_encoded[input_col]
y = df_encoded[output_col]
test_X = test_df_encoded[test_input_col]
# Initialize and fit
mean_model = MeanRegressor()
mean_model.fit(X, y)

# Predictions
y_pred = mean_model.predict(X)
print(y_pred[:5])


[180921.19589041 180921.19589041 180921.19589041 180921.19589041
 180921.19589041]


In [70]:
from sklearn.metrics import mean_squared_error
import numpy as np


In [71]:

mse = mean_squared_error(y, mean_model.predict(X))
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.2f}")


RMSE: 79415.29


# BaseLine models


In [72]:
from sklearn.linear_model import LinearRegression
linreg_model = LinearRegression()
linreg_model.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [73]:
train_preds = linreg_model.predict(X)
train_preds

array([206280.81712829, 197384.80385873, 209026.46920078, ...,
       274487.0229861 , 141892.58329065, 149330.95493474], shape=(1460,))

In [74]:
mse = mean_squared_error(y, train_preds)
rmse = np.sqrt(mse)
rmse

np.float64(19767.6047083042)

## 6. Feature Engineering

In [75]:
import pandas as pd

def analyze_column(df, col_name, freq_threshold=0.01):
    """
    Analyze a column for:
      - Rare categories (if categorical)
      - Numerical outliers (if numerical)

    Parameters:
        df (pd.DataFrame): The dataframe.
        col_name (str): The column name to analyze.
        freq_threshold (float): Minimum fraction (default 1%)
                                below which a category is 'rare'.

    Returns:
        dict: Contains:
              - 'type' → column type ('numerical' or 'categorical')
              - 'outliers' → list of outlier values (numerical)
              - 'rare_values' → list of rare categories (categorical)
              - 'details' → additional info (bounds, counts, etc.)
    """
    if col_name not in df.columns:
        print(f"❌ Column '{col_name}' not found.")
        return None

    col_data = df[col_name].dropna()

    # --------- Numerical Column ----------
    if pd.api.types.is_numeric_dtype(col_data):
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = col_data[(col_data < lower_bound) | (col_data > upper_bound)]

        result = {
            "type": "numerical",
            "outliers": outliers.tolist(),
            "details": {
                "count": len(outliers),
                "percentage": round((len(outliers) / len(col_data)) * 100, 2),
                "lower_bound": lower_bound,
                "upper_bound": upper_bound
            }
        }

        print(f"📊 '{col_name}' → Numerical column: {len(outliers)} outliers found "
              f"({result['details']['percentage']}%).")

    # --------- Categorical Column ----------
    else:
        freq = col_data.value_counts(normalize=True)
        rare_values = freq[freq < freq_threshold]

        result = {
            "type": "categorical",
            "rare_values": list(rare_values.index),
            "details": {
                "rare_value_counts": rare_values.to_dict(),
                "rare_count": len(rare_values),
                "threshold": freq_threshold
            }
        }

        if rare_values.empty:
            print(f"✅ '{col_name}' → No rare values found.")
        else:
            print(f"🔍 '{col_name}' → Found {len(rare_values)} rare values "
                  f"(below {freq_threshold*100}% frequency).")

    return result


In [76]:
result_num = analyze_column(data_df, 'Fireplaces')
print(result_num['outliers'][:10])

📊 'Fireplaces' → Numerical column: 5 outliers found (0.34%).
[3, 3, 3, 3, 3]


In [77]:
## function to drop outlier
def drop_outlier_rows(df, col_name, analysis_result):
    """
    Drops rows from a DataFrame that contain outlier values
    based on the analyze_column() output.

    Parameters:
        df (pd.DataFrame): The original DataFrame.
        col_name (str): Column name where outliers were detected.
        analysis_result (dict): Output from analyze_column() function.

    Returns:
        pd.DataFrame: A new DataFrame with outlier rows removed.
    """
    if not analysis_result or analysis_result.get("type") != "numerical":
        print("❌ The provided result does not contain numerical outlier data.")
        return df
    
    outlier_values = analysis_result.get("outliers", [])
    
    if not outlier_values:
        print(f"✅ No outliers found in '{col_name}'. Nothing to drop.")
        return df
    
    # Drop rows where column value is one of the outliers
    initial_count = len(df)
    df_cleaned = df[~df[col_name].isin(outlier_values)].copy()
    removed_count = initial_count - len(df_cleaned)
    
    print(f"🧹 Removed {removed_count} outlier rows from '{col_name}'.")
    
    return df_cleaned


In [78]:
drop_outlier_rows(data_df, 'YearBuilt', result_num)

🧹 Removed 0 outlier rows from 'YearBuilt'.


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,61,0,0,0,NA,NA,NA,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,0,0,0,NA,NA,NA,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,42,0,0,0,NA,NA,NA,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,35,272,0,0,NA,NA,NA,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,84,0,0,0,NA,NA,NA,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,Inside,...,40,0,0,0,NA,NA,NA,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,0,0,0,NA,MnPrv,NA,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,Inside,...,60,0,0,0,NA,GdPrv,Shed,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,112,0,0,NA,NA,NA,WD,Normal,142125


In [79]:
df_encoded = one_hot_encode_all(train_df, categorical_cols)


✅ One-hot encoding applied to 46 columns.
🔢 Data shape changed from (1460, 72) → (1460, 326)


In [80]:
df_encoded.shape

(1460, 326)

In [81]:
X = df_encoded[input_col]
Y = df_encoded[output_col]
test_X = test_df_encoded[test_input_col]

$$Random Forest

In [82]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor(max_depth=20, n_jobs=1, random_state=42, n_estimators=26)
model2.fit(X, Y)

,n_estimators,26
,criterion,'squared_error'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [83]:
mse = mean_squared_error(Y, model2.predict(X))
rmse = np.sqrt(mse)
rmse

np.float64(11697.477983096836)

$$Gradient Boosting


In [84]:
!pip install xgboost
from xgboost import XGBRegressor
model3 = XGBRegressor(random_state=42, n_jobs=1,learning_rate = 0.8, objective='reg:squarederror' , eval_metric = 'rmse')
model3.fit(X, Y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'rmse'


In [85]:
mse = mean_squared_error(Y, model3.predict(X))
rmse = np.sqrt(mse)
rmse

np.float64(185.7307043308941)

```Prepare Training DF ```

In [86]:
test_df = pd.read_csv(f'{data_dir}/test.csv', nrows=100000)

In [87]:
test_df = test_df.drop(test_remove_cols, axis=1)
test_df = clean_missing_values(test_df)
area = most_Val(test_df, 'MasVnrArea')
area
test_df['MasVnrArea'].fillna(area, inplace=True)
area = most_Val(test_df, 'LotFrontage')
area
test_df['LotFrontage'].fillna(area, inplace=True)

/tmp/ipykernel_7171/2073745228.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_7171/2073745228.py:16: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' o

In [88]:
test_df_encoded = one_hot_encode_all(test_df, categorical_cols)
test_df_encoded.shape


✅ One-hot encoding applied to 46 columns.
🔢 Data shape changed from (1459, 72) → (1459, 315)


(1459, 315)

In [89]:
test_X = test_df_encoded[test_input_col]

In [90]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Step 5: Ensure columns match model3's training data
# If your model3 was trained on a DataFrame `train_X`
# Align both to have same features
try:
    test_X = test_df.reindex(columns=model3.feature_names_in_, fill_value=0)
except AttributeError:
    # In case model3 doesn't have feature_names_in_ attribute
    test_X = test_df.copy()

# Step 6: Predict SalePrice using model3
predictions = model3.predict(test_X)

# Step 7: Create submission DataFrame
submission = pd.DataFrame({
    "Id": test_df["Id"],   # use Id column from test.csv
    "SalePrice": predictions
})

# Step 8: Save to CSV
submission.to_csv("data/submission.csv", index=False)
print("✅ submission.csv created successfully!")

# Optional preview
submission.head()


✅ submission.csv created successfully!


,Id,SalePrice
0,1461,132352.421875
1,1462,116106.320312
2,1463,157930.890625
3,1464,168901.000000
4,1465,138775.718750
